

# **CS412 - Machine Learning**
### Term Project

*Credit Risk Analysis with the German Credit Data*

Arda Baydar

Baris Baran Aydogan - 28425

Bukey Yurt - 28263


In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import random as rn
import tensorflow as tf
from sklearn.model_selection import train_test_split
seed = 42
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/412_project/german_credit_data.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.head(11)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,1,free,NaN,NaN,9055,36,education,good
6,53,male,2,own,quite rich,NaN,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,NaN,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [ ]:
most_common_CA = df['Checking account'].mode()[0]
most_common_SA = df['Saving accounts'].mode()[0]
print(most_common_CA)
print(most_common_SA)

# Ekstra Feature Engineering
df['BBAIndex'] = (df['Credit amount']*df['Age']) / df['Duration']





little
little


In [ ]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,BBAIndex
0,67,male,2,own,NaN,little,1169,6,radio/TV,good,13053.833333
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad,2727.541667
2,49,male,1,own,little,NaN,2096,12,education,good,8558.666667
3,45,male,2,free,little,little,7882,42,furniture/equipment,good,8445.000000
4,53,male,2,free,little,little,4870,24,car,bad,10754.583333
...,...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good,4484.666667
996,40,male,3,own,little,little,3857,30,car,good,5142.666667
997,38,male,2,own,little,NaN,804,12,radio/TV,good,2546.000000
998,23,male,2,free,little,little,1845,45,radio/TV,bad,943.000000


In [ ]:
print(df.isna().sum())

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
BBAIndex              0
dtype: int64


In [ ]:
df = df.fillna("little")

In [ ]:
print(df.isna().sum())

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
BBAIndex            0
dtype: int64


In [ ]:
categorical_columns = ['Sex','Housing','Saving accounts','Checking account', 'Purpose']

df_encoded = pd.get_dummies(df, columns=categorical_columns)
df_encoded['Risk'] = df_encoded['Risk'].replace({'bad':0 , 'good': 1})
df_encoded['Age_Category'] = pd.qcut(df_encoded['Age'], q=4, labels=False)
df_encoded['Duration_Category'] = pd.qcut(df_encoded['Duration'], q=4, labels=False)

In [ ]:
df_encoded.head()

,Age,Job,Credit amount,Duration,Risk,BBAIndex,Sex_female,Sex_male,Housing_free,Housing_own,...,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Age_Category,Duration_Category
0,67,2,1169,6,1,13053.833333,0,1,0,1,...,0,0,0,0,0,1,0,0,3,0
1,22,2,5951,48,0,2727.541667,1,0,0,1,...,0,0,0,0,0,1,0,0,0,3
2,49,1,2096,12,1,8558.666667,0,1,0,1,...,0,0,0,1,0,0,0,0,3,0
3,45,2,7882,42,1,8445.000000,0,1,1,0,...,0,0,0,0,1,0,0,0,3,3
4,53,2,4870,24,0,10754.583333,0,1,1,0,...,0,1,0,0,0,0,0,0,3,2


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

X = df_encoded.drop('Risk', axis=1)
y = df_encoded['Risk']

# Split the data into 70% training, 15% validation, and 15% test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)



Training set shape: (700, 27)
Validation set shape: (150, 27)
Test set shape: (150, 27)


In [ ]:
corr = df_encoded.corr(method = 'pearson')
corr =corr.sort_values("Risk",ascending=False)
corr

,Age,Job,Credit amount,Duration,Risk,BBAIndex,Sex_female,Sex_male,Housing_free,Housing_own,...,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Age_Category,Duration_Category
Risk,0.091127,-0.032735,-0.154739,-0.214927,1.000000,-0.011426,-0.075493,0.075493,-0.081556,0.134589,...,-0.036129,-0.022621,-0.008016,-0.049085,-0.020971,0.106922,-0.020828,-0.028058,0.108076,-0.178140
Housing_own,0.006553,-0.059393,-0.117497,-0.075169,0.134589,-0.042972,-0.119638,0.119638,-0.548445,1.000000,...,0.051083,-0.099526,0.029317,-0.075684,-0.040496,0.134705,0.019803,-0.011288,0.026859,-0.044367
Age_Category,0.903122,0.043132,0.037802,-0.032311,0.108076,0.302433,-0.210148,0.210148,0.245508,0.026859,...,-0.019334,0.116600,-0.025291,0.051237,-0.115105,-0.048581,0.021456,0.039227,1.000000,-0.040188
Purpose_radio/TV,-0.046401,-0.027683,-0.173203,-0.044319,0.106922,-0.119007,-0.008668,0.008668,-0.109357,0.134705,...,-0.204388,-0.444602,-0.068727,-0.156151,-0.293164,1.000000,-0.093531,-0.068727,-0.048581,-0.056522
Age,1.000000,0.015673,0.032716,-0.036136,0.091127,0.362912,-0.161694,0.161694,0.253058,0.006553,...,-0.001772,0.099919,-0.012562,0.060390,-0.127657,-0.046401,0.039567,0.042365,0.903122,-0.051892
Checking account_little,0.054212,0.029670,-0.060775,-0.044786,0.089895,0.011343,-0.014141,0.014141,-0.007827,-0.010721,...,-0.091809,0.057879,0.019190,-0.003713,0.055659,-0.028565,-0.024552,-0.058817,0.067307,-0.016347
Saving accounts_rich,0.032007,-0.017128,-0.055542,-0.048261,0.085749,-0.038394,0.041673,-0.041673,-0.047989,0.028707,...,0.021244,0.018052,-0.024747,-0.016518,0.028091,-0.046259,0.030106,-0.024747,0.038844,-0.026119
Sex_male,0.161694,0.070298,0.093482,0.081432,0.075493,0.097656,-1.000000,1.000000,0.100872,0.119638,...,0.080875,0.047893,-0.045275,-0.052397,-0.100467,0.008668,0.026828,0.014297,0.210148,0.076365
Saving accounts_quite rich,0.030988,-0.037499,-0.064256,-0.040257,0.070954,-0.026440,-0.031414,0.031414,-0.010662,0.000737,...,-0.029357,-0.028134,0.047023,-0.012524,0.006382,0.049134,-0.010831,-0.028577,0.043501,-0.029508
Checking account_rich,0.037504,-0.050099,-0.100510,-0.076455,0.044009,-0.035890,0.004183,-0.004183,0.015860,0.018934,...,-0.015451,-0.028134,0.009223,0.004943,-0.004308,0.058300,-0.038890,-0.028577,0.029044,-0.067808


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Define parameter grid for grid search
param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}

# Instantiate KNN classifier and perform grid search for hyperparameter tuning
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print best hyperparameters and their scores
print('Best hyperparameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

# Train final model using best hyperparameters and all training data
best_knn = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'],
                                 weights=grid_search.best_params_['weights'])
best_knn.fit(X_train, y_train)

# Evaluate final model on test set
y_pred = best_knn.predict(X_test)
print(classification_report(y_test, y_pred))
test_accuracy=accuracy_score(y_test,y_pred)*100

y_val_pred = best_knn.predict(X_val)
print(classification_report(y_val, y_val_pred))
val_accuracy=accuracy_score(y_val,y_val_pred)*100

print("Accuracy for our Validation dataset with tuning is : {:.2f}%".format(val_accuracy) )
print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy) )

Best hyperparameters:  {'n_neighbors': 9, 'weights': 'uniform'}
Best score:  0.7028571428571428
              precision    recall  f1-score   support

           0       0.48      0.25      0.33        40
           1       0.77      0.90      0.83       110

    accuracy                           0.73       150
   macro avg       0.62      0.57      0.58       150
weighted avg       0.69      0.73      0.69       150

              precision    recall  f1-score   support

           0       0.41      0.14      0.21        51
           1       0.67      0.90      0.77        99

    accuracy                           0.64       150
   macro avg       0.54      0.52      0.49       150
weighted avg       0.58      0.64      0.58       150

Accuracy for our Validation dataset with tuning is : 64.00%
Accuracy for our testing dataset with tuning is : 72.67%


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = {'max_depth': [3, 5, 7], 'min_samples_leaf': [2, 4, 6]}

# Instantiate Decision Tree classifier and perform grid search for hyperparameter tuning
dt = DecisionTreeClassifier()
grid_search = GridSearchCV(dt, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print best hyperparameters and their scores
print('Best hyperparameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

# Train final model using best hyperparameters and all training data
best_dt = DecisionTreeClassifier(max_depth=grid_search.best_params_['max_depth'],
                                  min_samples_leaf=grid_search.best_params_['min_samples_leaf'])
best_dt.fit(X_train, y_train)

# Evaluate final model on test set
y_pred = best_dt.predict(X_test)
print(classification_report(y_test, y_pred))
test_accuracy=accuracy_score(y_test,y_pred)*100

y_val_pred = best_dt.predict(X_val)
print(classification_report(y_val, y_val_pred))
val_accuracy=accuracy_score(y_val,y_val_pred)*100

print("Accuracy for our Validation dataset with tuning is : {:.2f}%".format(val_accuracy) )
print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy) )
best_DT= best_dt.score(X_train, y_train)
print("Best DT score : {:.2f}".format(best_DT) )


Best hyperparameters:  {'max_depth': 3, 'min_samples_leaf': 2}
Best score:  0.7142857142857142
              precision    recall  f1-score   support

           0       0.53      0.25      0.34        40
           1       0.77      0.92      0.84       110

    accuracy                           0.74       150
   macro avg       0.65      0.58      0.59       150
weighted avg       0.71      0.74      0.71       150

              precision    recall  f1-score   support

           0       0.48      0.20      0.28        51
           1       0.68      0.89      0.77        99

    accuracy                           0.65       150
   macro avg       0.58      0.54      0.52       150
weighted avg       0.61      0.65      0.60       150

Accuracy for our Validation dataset with tuning is : 65.33%
Accuracy for our testing dataset with tuning is : 74.00%
Best DT score : 0.77


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define parameter grid for grid search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],'penalty': ['l2']}

# Instantiate logistic regression classifier and perform grid search for hyperparameter tuning
lr = LogisticRegression(max_iter=10000, solver='lbfgs')
grid_search = GridSearchCV(lr, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters and their scores
print('Best hyperparameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

# Train final model using best hyperparameters and all training data
best_lr = LogisticRegression(C=grid_search.best_params_['C'], penalty=grid_search.best_params_['penalty'], max_iter=10000, solver='lbfgs')
best_lr.fit(X_train, y_train)

# Evaluate final model on test set
y_pred = best_lr.predict(X_test)
print(classification_report(y_test, y_pred))
test_accuracy=accuracy_score(y_test,y_pred)*100

y_val_pred = best_lr.predict(X_val)
print(classification_report(y_val, y_val_pred))
val_accuracy=accuracy_score(y_val,y_val_pred)*100

print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy) )
print("Accuracy for our Validation dataset with tuning is : {:.2f}%".format(val_accuracy) )
best_lr_score = best_lr.score(X_train, y_train)
print("Best LR score : {:.2f}".format(best_lr_score) )

Best hyperparameters:  {'C': 0.1, 'penalty': 'l2'}
Best score:  0.7157142857142857
              precision    recall  f1-score   support

           0       0.89      0.20      0.33        40
           1       0.77      0.99      0.87       110

    accuracy                           0.78       150
   macro avg       0.83      0.60      0.60       150
weighted avg       0.80      0.78      0.72       150

              precision    recall  f1-score   support

           0       0.44      0.16      0.23        51
           1       0.67      0.90      0.77        99

    accuracy                           0.65       150
   macro avg       0.56      0.53      0.50       150
weighted avg       0.60      0.65      0.59       150

Accuracy for our testing dataset with tuning is : 78.00%
Accuracy for our Validation dataset with tuning is : 64.67%
Best LR score : 0.73


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Create an AdaBoost classifier with the base classifier
adaboost_classifier = AdaBoostClassifier(best_lr, n_estimators=10, random_state=42)

# Train the AdaBoost classifier using the training set
adaboost_classifier.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = adaboost_classifier.predict(X_val)

# Evaluate the accuracy of the AdaBoost classifier on the validation set
validation_accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", validation_accuracy)

# Make predictions on the test set
y_pred_test = adaboost_classifier.predict(X_test)

# Evaluate the accuracy of the AdaBoost classifier on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)


best_Adaboost = adaboost_classifier.score(X_train, y_train)
print("Best AdaBoost score : {:.2f}".format(best_Adaboost) )

Validation Accuracy: 0.6933333333333334
Test Accuracy: 0.76
Best AdaBoost score : 0.71


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_test)
confusion_matrix

array([[  5,  35],
       [  1, 109]])